### COUNT()
- COUNT() returns a count of things. If you pass it the name of a column, it will return the number of entries in that column.

- COUNT() is an example of an aggregate function, which takes many values and returns one. (Other examples of aggregate functions include SUM(), AVG(), MIN(), and MAX().

### GROUP BY
GROUP BY takes the name of one or more columns, and treats all rows with the same value in that column as a single group when you apply aggregate functions like COUNT().

### Aliasing and other improvements
- 为聚集函数出来的新的一行取个别名用AS
- 如果不确定COUNT()里面什么参数，you can do COUNT(1) to count the rows in each group. Most people find it especially readable, because we know it's not focusing on other columns. It also scans less data than if supplied column names (making it faster and using less of your data access quota).

### ORDER BY
You can reverse the order using the DESC argument

### EXTRACT
Often you'll want to look at part of a date, like the year or the day. You can do this with EXTRACT. 比如说EXTRACT(DAY from Date)，EXTRACT(WEEK from Date)

There are two ways that dates can be stored in BigQuery: as a DATE or as a DATETIME.

- The DATE format has the year first, then the month: YYYY-[M]M-[D]D(One or two digit)
- DATETIME format is like the date format ... but with time added at the end.

In [ ]:
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql.ex4 import *
print("Setup Complete")

In [ ]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "world_bank_intl_education" dataset
dataset_ref = client.dataset("world_bank_intl_education", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "international_education" table
table_ref = dataset_ref.table("international_education")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the "international_education" table
client.list_rows(table, max_results=5).to_dataframe()

| | country_name  | country_code  | indicator_name  | indicator_code  | value  | year  |
|  ----  | ----  | ----  | ----  | ----  | ----  | ----  |
| 0  | Russian Federation 	 | RUS | Population of the official age for lower secon...| SP.SEC.LTOT.IN |6.710437e+06 |2016|
| 1  | Sao Tome and Principe 	|STP  | Percentage of students in pre-primary educatio... |SE.PRE.ENRL.FE.ZS|5.168527e+01|2016|
| 2 | Saudi Arabia  | SAU|Population, ages 15-64, female|SP.POP.1564.FE.IN|9.384885e+06|2016|
| 3  | Serbia | SRB |Duration of compulsory education (years)|SE.COM.DURS|8.000000e+00|2016|
| 4  | Serbia | SRB |Youth literacy rate, population 15-24 years, m..|SE.ADT.1524.LT.MA.ZS|9.971523e+01|2016|

# Exercises

The value in the `indicator_code` column describes what type of data is shown in a given row.  

One interesting indicator code is `SE.XPD.TOTL.GD.ZS`, which corresponds to "Government expenditure on education as % of GDP (%)".

### 1) Government expenditure on education

Which countries spend the largest fraction of GDP on education?  

To answer this question, consider only the rows in the dataset corresponding to indicator code `SE.XPD.TOTL.GD.ZS`, and write a query that returns the average value in the `value` column for each country in the dataset between the years 2010-2017 (including 2010 and 2017 in the average). 

Requirements:
- Your results should have the country name rather than the country code. You will have one row for each country.
- The aggregate function for average is **AVG()**.  Use the name `avg_ed_spending_pct` for the column created by this aggregation.
- Order the results so the countries that spend the largest fraction of GDP on education show up first.

In case it's useful to see a sample query, here's a query you saw in the tutorial (using a different dataset):
```
# Query to find out the number of accidents for each day of the week
query = """
        SELECT COUNT(consecutive_number) AS num_accidents, 
               EXTRACT(DAYOFWEEK FROM timestamp_of_crash) AS day_of_week
        FROM `bigquery-public-data.nhtsa_traffic_fatalities.accident_2015`
        GROUP BY day_of_week
        ORDER BY num_accidents DESC
        """
```

In [ ]:
country_spend_pct_query = """
                          SELECT country_name, AVG(value) AS avg_ed_spending_pct
                          FROM `bigquery-public-data.world_bank_intl_education.international_education`
                          WHERE indicator_code = 'SE.XPD.TOTL.GD.ZS' and year >= 2010 and year <= 2017
                          GROUP BY country_name
                          ORDER BY avg_ed_spending_pct DESC
                          """

# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
country_spend_pct_query_job = client.query(country_spend_pct_query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
country_spending_results = country_spend_pct_query_job.to_dataframe()

# View top few rows of results
print(country_spending_results.head())

| | country_name  | avg_ed_spending_pct  |
|  ----  | ----  | ----  |
| 0  | Cuba | 12.837270 |
| 1  | Micronesia, Fed. Sts.  | 12.467750 |
| 2 | Solomon Islands  | 10.001080|
| 3  | Moldova | 8.372153 |
| 4  | Namibia | 8.349610 |

### 2) Identify interesting codes to explore

The last question started by telling you to focus on rows with the code `SE.XPD.TOTL.GD.ZS`. But how would you find more interesting indicator codes to explore?

There are 1000s of codes in the dataset, so it would be time consuming to review them all. But many codes are available for only a few countries. When browsing the options for different codes, you might restrict yourself to codes that are reported by many countries.

Write a query below that selects the indicator code and indicator name for all codes with at least 175 rows in the year 2016.

Requirements:
- You should have one row for each indicator code.
- The columns in your results should be called `indicator_code`, `indicator_name`, and `num_rows`.
- Only select codes with 175 or more rows in the raw database (exactly 175 rows would be included).
- To get both the `indicator_code` and `indicator_name` in your resulting DataFrame, you need to include both in your **SELECT** statement (in addition to a **COUNT()** aggregation). This requires you to include both in your **GROUP BY** clause.
- Order from results most frequent to least frequent.

In [ ]:
# Your code goes here

code_count_query = """
                   SELECT indicator_code, indicator_name, COUNT(1) AS num_rows
                   FROM `bigquery-public-data.world_bank_intl_education.international_education`
                   WHERE year = 2016
                   GROUP BY indicator_name, indicator_code
                   HAVING COUNT(1) >= 175
                   ORDER BY COUNT(1) DESC
                   """

# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
code_count_query_job = client.query(code_count_query, job_config=safe_config)
code_count_results = code_count_query_job.to_dataframe()
print(code_count_results.head())

| | indicator_code  | indicator_name  | num_rows|
|  ----  | ----  | ----  | ----  |
| 0  | SP.POP.TOTL  | Population, total |232|
| 1  | SP.POP.TOTL | Population growth (annual %) |232|
| 2 | IT.NET.USER.P2   | Internet users (per 100 people)|223|
| 3  | SP.POP.0014.FE.IN | Population, ages 0-14, female |213|
| 4  | SP.POP.0014.TO.ZS | Population, ages 0-14 (% of total) |213|